# Logging {#sec-logging}

The REDCap log provides an audit trail to track record creation, deletion, update and export. It also tracks changes made to survey design and user rights.

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [1]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [2]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\CMD3303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


::: {.panel-tabset}

#### R

In [3]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [4]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [5]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [6]:
# import pycap
import redcap
import json
import pandas as pd
# import requests

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [7]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [8]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

::: {.panel-tabset}

#### R

`redcap_log_read()`

The `redcap_log_read` function is only available on REDCapR Version 1.1.9 and later.

In [12]:
%%capture
%%R
log <- redcap_log_read(redcap_uri = url, token)$data

::: {.content-hidden when-format="html"}

In [10]:
%%R
log_tbl <- gt(head(log))
gt::gtsave(log_tbl, filename = 'export_log.html', path = "./files/export_files/")

:::

<p align="center"><iframe width="100%" height="400" src="./files/export_files/export_log.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_logging()`

In [11]:
log = project.export_logging(format_type='df')
log.head()

,timestamp,username,action,details,record
0,2024-04-12 17:23,caitlin.drover@doh.wa.gov,Manage/Design,Export Logging (API),NaN
1,2024-04-12 17:22,caitlin.drover@doh.wa.gov,Manage/Design,Export instrument-event mappings (API),NaN
2,2024-04-12 17:22,caitlin.drover@doh.wa.gov,Data export (API),"export_format: CSV, rawOrLabel: raw, export_da...",NaN
3,2024-04-12 17:22,caitlin.drover@doh.wa.gov,Manage/Design,Download data dictionary (API),NaN
4,2024-04-12 17:22,caitlin.drover@doh.wa.gov,Manage/Design,Export User-DAG assignments (API),NaN


:::